# Generating the Dataset

In [366]:
import pandas as pd
import random
import copy
from sklearn import preprocessing

#Read in the csv of Naval Base information: Name, Lat, Lon, Number of Billets
naval_bases = pd.read_csv("naval_bases.csv")

# Run the normalize the number of billets for the desired number of simulated sailors
naval_bases["Normalized"] = naval_bases["Billets"] / naval_bases["Billets"].sum()

total_sailors = 1500
total_billets = 1500

naval_bases["Billets"] = naval_bases["Normalized"] * total_billets
naval_bases["Billets"] = naval_bases["Billets"].astype("int")

assigned_billets = naval_bases["Billets"].sum()
assigned_billets

1474

In [376]:
#Set up the sailor preferences
#Randomly assign each sailor 3 preferences wieghted by the number of billets at each locaiton
sailor_prefernces = []
for i in range(total_sailors):
    sailor_prefernces.append(list(naval_bases.Name.sample(3, replace=False, weights=naval_bases.Billets)))
    sailor_prefernces[i].insert(0,i)
sailor_prefernces = pd.DataFrame(sailor_prefernces, columns=["sailor_ID", "pref_1","pref_2", "pref_3"])
sailor_prefernces.head()
print (len(sailor_prefernces))

1500


In [368]:
#Set up the job owner prefences, these are a randomized ordering of all sailors wanting to go to that billet
job_owner_prefernces = []
for base in list(naval_bases.Name):
    first = sailor_prefernces.loc[sailor_prefernces['pref_1'] == base]
    second = sailor_prefernces.loc[sailor_prefernces['pref_2'] == base]
    third = sailor_prefernces.loc[sailor_prefernces['pref_3'] == base]
    sailors = pd.concat([first,second,third]).drop_duplicates("sailor_ID", keep="first")
    job_owner_prefernces.append([base,list(sailors.sailor_ID)])
job_owner_prefernces = pd.DataFrame(job_owner_prefernces,columns=["job_owner", "sailors"])
job_owner_prefernces.head()

,job_owner,sailors
0,CHINA LAKE NAVAL AIR WEAPONS STATION,"[9, 341, 419, 541, 893, 972, 1274, 1388, 188, ..."
1,CORONADO NAVAL AMPHIBIOUS BASE,"[5, 31, 97, 102, 148, 201, 276, 475, 506, 633,..."
2,EL CENTRO NAVAL AIR FACILITY,"[1298, 221]"
3,LEMOORE NAS,"[796, 1414, 1417, 1137, 1309, 1365, 669, 672, ..."
4,MONTEREY NAVAL POSTGRADUATE SCHOOL,"[401, 789, 809, 1082, 1252, 1477, 202, 673, 99..."


In [369]:
pref_1 = sailor_prefernces.groupby('pref_1').count()
pref_2 = sailor_prefernces.groupby('pref_2').count()
pref_3 = sailor_prefernces.groupby('pref_3').count()
naval_bases_hold = naval_bases.set_index("Name")
prefs = pd.concat([pref_1.sailor_ID, pref_2.sailor_ID,pref_3.sailor_ID,naval_bases_hold.Billets], axis=1)
prefs.columns = ["pref_1","pref_2","pref_3", "Billets"]
prefs.head()

,pref_1,pref_2,pref_3,Billets
ANNAPOLIS NS,18.0,25,15,21
BANGOR NAVAL SUBMARINE BASE,69.0,70,65,63
BEAUFORT NAVAL HOSPITAL,6.0,9,6,8
BETHESDA NATIONAL NAVAL MEDICAL CENTER,23.0,23,21,16
BREMERTON NAVAL STATION,14.0,30,23,21


In [370]:
#Shape dataframes into dictionaries for mathcing

#Sailors
sailor_prefernces_dict = sailor_prefernces.set_index('sailor_ID').T.to_dict('list')
sailor_prefernces_store = sailor_prefernces.set_index('sailor_ID').T.to_dict('list')

#Job Owners
job_owner_prefernces_dict = job_owner_prefernces.set_index('job_owner').T.to_dict('records')[0]

#Capacities
billets = pd.Series(naval_bases.Billets.values,index=naval_bases.Name).to_dict()

#Shape dataframes for graphing
#Billets
graph_bases = naval_bases.loc[naval_bases.index.repeat(naval_bases.Billets)]

In [165]:
import folium
from folium import plugins

m = folium.Map([41.8781, -87.6298], zoom_start=4)

# convert to (n, 2) nd-array format for heatmap
billet_loc = graph_bases[['Lat', 'Lon']].as_matrix()
# Add AOI to the markers***
billet_info = graph_bases["Name"].as_matrix()

# plot heatmap
hm = plugins.HeatMap(billet_loc.tolist(), name="Billet HeatMap")
m.add_child(hm)

# plot markers
hm = plugins.MarkerCluster(locations = billet_loc.tolist(), popups = billet_info.tolist(), name="Billets")
m.add_child(hm)

#Add Layer Control
# add the layer control
folium.LayerControl().add_to(m)


m

In [371]:
from collections import Counter
from copy import deepcopy

import numpy as np

def check_inputs(hospital_prefs, resident_prefs):
    """ Reduce as necessary the preference list of all residents and hospitals
    so that no player ranks another player that they are not also ranked by. """

    for resident in resident_prefs.keys():
        for hospital in resident_prefs[resident]:
            if resident not in hospital_prefs[hospital]:
                raise ValueError(
                    'Hospitals must rank all residents who rank them.'
                )


def get_free_residents(resident_prefs, matching):
    """ Return a list of all residents who are currently unmatched but have a
    non-empty preference list. """

    return [
        resident
        for resident in resident_prefs
        if resident_prefs[resident]
        and not any([resident in match for match in matching.values()])
    ]


def get_worst_idx(hospital, hospital_prefs, matching):
    """ Find the index of the worst resident currently assigned to `hospital`
    according to their preferences. """

    return max(
        [
            hospital_prefs[hospital].index(resident)
            for resident in hospital_prefs[hospital]
            if resident in matching[hospital]
        ]
    )


def hospital_resident(hospital_prefs, resident_prefs, capacities):
    """ Provide a stable, resident-optimal matching for the given instance of
    HR using the algorithm set out in [Dubins, Freeman 1981]. """

    check_inputs(hospital_prefs, resident_prefs)

    matching = {hospital: [] for hospital in hospital_prefs}
    free_residents = get_free_residents(resident_prefs, matching)
    count = 0
    while free_residents:
        count += 1
        resident = free_residents[0]
        hospital = resident_prefs[resident][0]
        matching[hospital].append(resident)
        if len(matching[hospital]) > capacities[hospital]:
            worst = get_worst_idx(hospital, hospital_prefs, matching)
            resident = hospital_prefs[hospital][worst]
            matching[hospital].remove(resident)

        if len(matching[hospital]) == capacities[hospital]:
            worst = get_worst_idx(hospital, hospital_prefs, matching)
            successors = hospital_prefs[hospital][worst + 1:]

            if successors:
                for resident in successors:
                    count += 1
                    hospital_prefs[hospital].remove(resident)
                    if hospital in resident_prefs[resident]:
                        resident_prefs[resident].remove(hospital)

        free_residents = get_free_residents(resident_prefs, matching)

    for hospital, matches in matching.items():
        sorted_matches = sorted(matches, key=hospital_prefs[hospital].index)
        matching[hospital] = sorted_matches
    print (count)
    return matching

match = hospital_resident(job_owner_prefernces_dict, sailor_prefernces_dict, billets)

3541


In [372]:
#verify Results and Show Preferences
sailor_results = []
for base, sailors in match.items():
    for member in sailors:
        prefernce = sailor_prefernces_store[member].index(base)
        sailor_results.append([member, prefernce + 1])
results = pd.DataFrame(sailor_results, columns=["Sailor", "preference_received"])
results.groupby('preference_received').count()

,Sailor
preference_received,
1,1406
2,29
3,12


In [373]:
#Data Verification
flat_list = pd.Series([item for sublist in list(match.values()) for item in sublist])
print(flat_list.value_counts().max())
print(len(flat_list.values) - assigned_billets)

1
-27


In [374]:
len(flat_list.values)


1447

In [375]:
assigned_billets

1474